# **Importamos librerias**

In [1]:
# manejo de data

import pandas as pd
import numpy as np
from sklearn import preprocessing
import joblib

# conexion con el drive

from google.colab import drive



#**Conectamos al Drive**

In [2]:

# conectamos al drive

drive.mount('/content/drive')

Mounted at /content/drive


#**Importamos la data Categorica**

In [3]:

# Dfinimos el directorio

dir_data='./drive/MyDrive/springleaf/data'


# descargamos unicamente la data que debe ser categorizada
# por asunto de rendimiento del notebook


df_train_cat=pd.read_csv(
                filepath_or_buffer=dir_data+'/train.csv').drop(
                                                ['ID', 'target'],
                                                    axis=1).fillna(value=-1).select_dtypes(include=['object'])

df_test_cat=pd.read_csv(
                filepath_or_buffer=dir_data+'/test.csv').drop(
                                                ['ID'],
                                                    axis=1).fillna(value=-1).select_dtypes(include=['object'])



<ipython-input-3-6346243ecc20>:10: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_cat=pd.read_csv(
<ipython-input-3-6346243ecc20>:15: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test_cat=pd.read_csv(


# **Preparamos el preprocesado Categorico**

In [4]:
# vamos a generar las clases categoricas-ordinales y normalizar los numericos

pre_proc_list=[]

# para el conjunto de entrenamiento

for k in df_train_cat.columns:

  lbl = preprocessing.LabelEncoder()

  lbl.fit( list(df_train_cat[k].values) + list(df_test_cat[k].values) )

  df_train_cat[k] = lbl.transform(  list(df_train_cat[k].values)    )

  pre_proc_list.append(lbl)

# **Procesamiento Numerico**

In [5]:

# obtenemos los indices para dividir la data para el entrenamiento


# generamos los datasets numericos

df_train_num=(pd.read_csv(
                filepath_or_buffer=dir_data+'/train.csv'
                        )).drop(
                                      ['ID', 'target'],axis=1).fillna(
                                                                      value=-1).select_dtypes(include=['int','float'])


<ipython-input-5-877d99b3f4bf>:6: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_num=(pd.read_csv(


In [6]:
# obtenemos los indices para dividir la data

mask_length=df_train_cat.shape[0]

mask_index=np.random.rand(mask_length)

# diividimos la data etiquetada en 60% train y 40% test

train_index=mask_index<=0.6

test_index=mask_index >0.6

In [7]:
# definimos la transformacion numerica

scaler=preprocessing.StandardScaler()

scaler.fit(
    pd.concat(
        [df_train_num[train_index],df_train_cat[train_index]],
        axis=1,
        ignore_index=True)
                              )

StandardScaler()

# **Exportamos la informacion de interes**

In [8]:


# extraemos los dos dataframes transformados

# para los features


np.save(dir_data+'/df_X_train.npy',scaler.transform( pd.concat(
        [df_train_num[train_index],df_train_cat[train_index]],
        axis=1,
        ignore_index=True)))

np.save(dir_data+'/df_X_test.npy',scaler.transform( pd.concat(
        [df_train_num[test_index],df_train_cat[test_index]],
        axis=1,
        ignore_index=True)))


# limpiamos la ram

del df_train_cat, df_train_num, df_test_cat

# para los targets

np.save(dir_data+'/df_y_train.npy',pd.read_csv(filepath_or_buffer=dir_data+'/train.csv')['target'][train_index])

np.save(dir_data+'/df_y_test.npy',pd.read_csv(filepath_or_buffer=dir_data+'/train.csv')['target'][test_index])

# nos llevamos las funciones de preprocesado

joblib.dump(pre_proc_list, dir_data+'/pre_pipeline.sav')

joblib.dump(scaler,dir_data+'/scaler.sav')


# fin



<ipython-input-8-6a8f11f2513a>:23: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  np.save(dir_data+'/df_y_train.npy',pd.read_csv(filepath_or_buffer=dir_data+'/train.csv')['target'][train_index])
<ipython-input-8-6a8f11f2513a>:25: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  np.save(dir_data+'/df_y_test.npy',pd.read_csv(filepath_or_buffer=dir_data+'/train.csv')['target'][test_index])


['./drive/MyDrive/springleaf/data/scaler.sav']